In [70]:
def loadDataSet():
    """
    创建数据集
    :return: 单词列表postingList, 所属类别classVec
    """
    postingList = [['my', 'my', 'has', 'flea', 'problems', 'help', 'please'], #[0,0,1,1,1......]
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  # 1 is abusive, 0 not
    return postingList, classVec

In [3]:
def createVocabList(dataSet):
    """
    获取所有单词的集合
    :param dataSet: 数据集
    :return: 所有单词的集合(即不含重复元素的单词列表)
    """
    vocabSet = set([])  # create empty set
    for document in dataSet:
        # 操作符 | 用于求两个集合的并集
        vocabSet = vocabSet | set(document)  # union of the two sets
    return list(vocabSet)


In [5]:
def setOfWords2Vec(vocabList, inputSet):
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocabList: 所有单词集合列表
    :param inputSet: 输入数据集
    :return: 匹配列表[0,1,0,1...]，其中 1与0 表示词汇表中的单词是否出现在输入的数据集中
    """
    # 创建一个和词汇表等长的向量，并将其元素都设置为0
    returnVec = [0] * len(vocabList)# [0,0......]
    # 遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [17]:
from numpy import *

In [18]:
def _trainNB0(trainMatrix, trainCategory):
    """
    训练数据原版
    :param trainMatrix: 文件单词矩阵 [[1,0,1,1,1....],[],[]...]
    :param trainCategory: 文件对应的类别[0,1,1,0....]，列表长度等于单词矩阵数，其中的1代表对应的文件是侮辱性文件，0代表不是侮辱性矩阵
    :return:
    """
    # 文件数
    numTrainDocs = len(trainMatrix)
    # 单词数
    numWords = len(trainMatrix[0])
    # 侮辱性文件的出现概率，即trainCategory中所有的1的个数，
    # 代表的就是多少个侮辱性文件，与文件的总数相除就得到了侮辱性文件的出现概率
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 构造单词出现次数列表
    p0Num = zeros(numWords) # [0,0,0,.....]
    p1Num = zeros(numWords) # [0,0,0,.....]

    # 整个数据集单词出现总数
    p0Denom = 0.0
    p1Denom = 0.0
    for i in range(numTrainDocs):
        # 是否是侮辱性文件
        if trainCategory[i] == 1:
            # 如果是侮辱性文件，对侮辱性文件的向量进行加和
            p1Num += trainMatrix[i] #[0,1,1,....] + [0,1,1,....]->[0,2,2,...]
            # 对向量中的所有元素进行求和，也就是计算所有侮辱性文件中出现的单词总数
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 类别1，即侮辱性文档的[P(F1|C1),P(F2|C1),P(F3|C1),P(F4|C1),P(F5|C1)....]列表
    # 即 在1类别下，每个单词出现的概率
    p1Vect = p1Num / p1Denom# [1,2,3,5]/90->[1/90,...]
    # 类别0，即正常文档的[P(F1|C0),P(F2|C0),P(F3|C0),P(F4|C0),P(F5|C0)....]列表
    # 即 在0类别下，每个单词出现的概率
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive

In [20]:
listOPosts, listClasses = loadDataSet()
myVocabList =createVocabList(listOPosts)

In [21]:
listOPosts

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [33]:
myVocabList

['dalmation',
 'love',
 'posting',
 'help',
 'mr',
 'maybe',
 'worthless',
 'please',
 'my',
 'buying',
 'dog',
 'cute',
 'quit',
 'so',
 'ate',
 'park',
 'not',
 'stupid',
 'problems',
 'take',
 'has',
 'how',
 'I',
 'garbage',
 'to',
 'licks',
 'food',
 'flea',
 'steak',
 'stop',
 'is',
 'him']

In [27]:
train_mat = []
for post_in in listOPosts:
    train_mat.append(setOfWords2Vec(myVocabList, post_in))

In [35]:
import numpy as np

In [36]:
np.array(train_mat)

array([[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [29]:
pov,p1v,pAb=_trainNB0(train_mat, listClasses)

In [30]:
pAb

0.5

In [31]:
pov

array([0.04166667, 0.04166667, 0.        , 0.04166667, 0.04166667,
       0.        , 0.        , 0.04166667, 0.125     , 0.        ,
       0.04166667, 0.04166667, 0.        , 0.04166667, 0.04166667,
       0.        , 0.        , 0.        , 0.04166667, 0.        ,
       0.04166667, 0.04166667, 0.04166667, 0.        , 0.04166667,
       0.04166667, 0.        , 0.04166667, 0.04166667, 0.04166667,
       0.04166667, 0.08333333])

In [32]:
p1v

array([0.        , 0.        , 0.05263158, 0.        , 0.        ,
       0.05263158, 0.10526316, 0.        , 0.        , 0.05263158,
       0.10526316, 0.        , 0.05263158, 0.        , 0.        ,
       0.05263158, 0.05263158, 0.15789474, 0.        , 0.05263158,
       0.        , 0.        , 0.        , 0.05263158, 0.05263158,
       0.        , 0.05263158, 0.        , 0.        , 0.05263158,
       0.        , 0.05263158])

In [46]:
def trainNB(train_mat, train_category):

    """

    朴素贝叶斯分类修正版，　注意和原来的对比，为什么这么做可以查看书

    :param train_mat:  type is ndarray

                    总的输入文本，大致是 [[0,1,0,1], [], []]

    :param train_category: 文件对应的类别分类， [0, 1, 0],

                            列表的长度应该等于上面那个输入文本的长度

    :return: 

    """

    train_doc_num = len(train_mat)

    words_num = len(train_mat[0])

    # 因为侮辱性的被标记为了1， 所以只要把他们相加就可以得到侮辱性的有多少

    # 侮辱性文件的出现概率，即train_category中所有的1的个数，

    # 代表的就是多少个侮辱性文件，与文件的总数相除就得到了侮辱性文件的出现概率

    pos_abusive = np.sum(train_category) / train_doc_num

    # 单词出现的次数

    # 原版，变成ones是修改版，这是为了防止数字过小溢出

    # p0num = np.zeros(words_num)

    # p1num = np.zeros(words_num)

    p0num = np.ones(words_num)

    p1num = np.ones(words_num)

    # 整个数据集单词出现的次数（原来是0，后面改成2了）

    p0num_all = 2.0

    p1num_all = 2.0



    for i in range(train_doc_num):

        # 遍历所有的文件，如果是侮辱性文件，就计算此侮辱性文件中出现的侮辱性单词的个数

        if train_category[i] == 1:

            p1num += train_mat[i]

            p1num_all += np.sum(train_mat[i])

        else:

            p0num += train_mat[i]

            p0num_all += np.sum(train_mat[i])

    # 后面改成取 log 函数

    p1vec = np.log(p1num / p1num_all)

    p0vec = np.log(p0num / p0num_all)

    return p0vec, p1vec, pos_abusive

In [67]:
def classifyNB(vec2classify, p0vec, p1vec, p_class1):

    """

    使用算法：

        # 将乘法转换为加法

        乘法：P(C|F1F2...Fn) = P(F1F2...Fn|C)P(C)/P(F1F2...Fn)

        加法：P(F1|C)*P(F2|C)....P(Fn|C)P(C) -> log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))

    :param vec2classify: 待测数据[0,1,1,1,1...]，即要分类的向量

    :param p0vec: 类别0，即正常文档的[log(P(F1|C0)),log(P(F2|C0)),log(P(F3|C0)),log(P(F4|C0)),log(P(F5|C0))....]列表

    :param p1vec: 类别1，即侮辱性文档的[log(P(F1|C1)),log(P(F2|C1)),log(P(F3|C1)),log(P(F4|C1)),log(P(F5|C1))....]列表

    :param p_class1: 类别1，侮辱性文件的出现概率

    :return: 类别1 or 0

    """

    # 计算公式  log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))

    # 使用 NumPy 数组来计算两个向量相乘的结果，这里的相乘是指对应元素相乘，即先将两个向量中的第一个元素相乘，然后将第2个元素相乘，以此类推。

    # 我的理解是：这里的 vec2Classify * p1Vec 的意思就是将每个词与其对应的概率相关联起来

    # 可以理解为 1.单词在词汇表中的条件下，文件是good 类别的概率 也可以理解为 2.在整个空间下，文件既在词汇表中又是good类别的概率
    print("vec2classify=%s" %vec2classify)
    print("p0vec=%s" %p0vec)
    print("p1vec=%s" %p1vec)
    print("p_class1=%s" %p_class1)
    p1 = np.sum(vec2classify * p1vec) + np.log(p_class1)
    print("p0=%s" %p1)
    p0 = np.sum(vec2classify * p0vec) + np.log(1 - p_class1)
    print("p1=%s" %p0)
    if p1 > p0:

        return 1

    else:

        return 0

In [71]:
def testingNB():

    """

    测试朴素贝叶斯算法

    :return: no return 

    """

    # 1. 加载数据集

    list_post, list_classes = loadDataSet()
    print("list_post=%s" %list_post)
    print("list_classes=%s" %list_classes)
    # 2. 创建单词集合

    vocab_list = createVocabList(list_post)
    print("vocab_list=%s" %vocab_list)


    # 3. 计算单词是否出现并创建数据矩阵

    train_mat = []

    for post_in in list_post:

        train_mat.append(

            # 返回m*len(vocab_list)的矩阵， 记录的都是0，1信息

            # 其实就是那个东西的句子向量（就是data_set里面每一行,也不算句子吧)

            setOfWords2Vec(vocab_list, post_in)

        )
    print("train_mat=%s" %train_mat)
    # 4. 训练数据

    p0v, p1v, p_abusive = trainNB(np.array(train_mat), np.array(list_classes))
    # 5. 测试数据

    test_one = ['love', 'my', 'dalmation']
    test_one_doc = np.array(setOfWords2Vec(vocab_list, test_one))
    print("test_one_doc=%s" %test_one_doc)
    print('the result is: {}'.format(classifyNB(test_one_doc, p0v, p1v, p_abusive)))

    test_two = ['stupid', 'garbage']

    test_two_doc = np.array(setOfWords2Vec(vocab_list, test_two))

    print('the result is: {}'.format(classifyNB(test_two_doc, p0v, p1v, p_abusive)))

In [72]:
 testingNB()

list_post=[['my', 'my', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
list_classes=[0, 1, 0, 1, 0, 1]
vocab_list=['dalmation', 'love', 'posting', 'help', 'mr', 'maybe', 'worthless', 'please', 'my', 'buying', 'dog', 'cute', 'quit', 'so', 'ate', 'park', 'not', 'stupid', 'problems', 'take', 'has', 'how', 'I', 'garbage', 'to', 'licks', 'food', 'flea', 'steak', 'stop', 'is', 'him']
train_mat=[[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 1, 

In [73]:
def BagOfWords2VecMN(vocabList, inputSet):
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocabList: 所有单词集合列表
    :param inputSet: 输入数据集
    :return: 匹配列表[0,1,0,1...]，其中 1与0 表示词汇表中的单词是否出现在输入的数据集中
    """
    # 创建一个和词汇表等长的向量，并将其元素都设置为0
    returnVec = [0] * len(vocabList)# [0,0......]
    # 遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec